# Basics of Flask:

In [ ]:
from flask import Flask,render_template,request

app = Flask(__name__)

@app.route('/')
def greet():
    return "Hello World!"

if __name__ == '__main__':
    app.run(host='0.0.0.0')

2. Build a Flask app with static HTML pages and navigate between them.

In [ ]:
from flask import Flask,render_template,request

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')


@app.route('/choose', methods=['POST'])
def choose():
    x = request.form.get('page')
    if x == 'about':
        return render_template('about.html')
    elif x == 'contact':
        return render_template('contact.html')
    elif x == 'blog':
        return render_template('blog.html')
    elif x == 'home':
        return render_template('index.html')  
    else:
        return render_template('index.html')  

if __name__ == '__main__':
    app.run(host='0.0.0.0',port='8000')

3. Develop a Flask app that uses URL parameters to display dynamic content.

In [ ]:
from flask import Flask,render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/user/<username>')
def showProfile(username):
    return render_template('profile.html',username = username)


if __name__ == '__main__':
    app.run(host='0.0.0.0',port='8000')

4. Create a Flask app with a form that accepts user input and displays it.

In [ ]:
from flask import Flask,render_template,request

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/display',methods=['POST'])
def showProfile():
    name = request.form.get("name")
    return render_template('display.html',name = name)


if __name__ == '__main__':
    app.run(host='0.0.0.0',port='8000')

5. Implement user sessions in a Flask app to store and display user-specific data.

In [ ]:
from flask import Flask,render_template,request,session,url_for,redirect
import os

app = Flask(__name__)
app.secret_key = os.urandom(24)

@app.route('/')
def home():
    username = session.get('username', None)
    if username:
        username = session['username']
        return render_template('welcome.html',username = username)
    else:
        return render_template('index.html')


@app.route('/login',methods=['POST'])
def login():
    username = request.form.get('username')
    session['username'] = username
    return redirect(url_for('home'))


@app.route('/logout')
def logout():
    session.pop('username',None)
    return redirect(url_for('home'))




if __name__ == '__main__':
    app.run(host='0.0.0.0',port='8000')

# Intermediate Flask Topics:

In [ ]:
6. Build a Flask app that allows users to upload files and display them on the website.

In [ ]:
from flask import Flask,render_template,request,redirect,url_for
import os

app = Flask(__name__)

uploadFolder = 'uploads'
app.config['uploadFolder'] = uploadFolder

if not os.path.exists(uploadFolder):
    os.makedirs(uploadFolder)

@app.route('/')
def index():
    files = os.listdir(uploadFolder)
    return render_template('index.html',files = files)

@app.route('/upload', methods = ['POST'])
def upload():
    if 'files' not in request.files:
        return redirect(request.url)
    
    file = request.files['files']

    if file.filename=='':
        return redirect(request.url)

    file.save(os.path.join(app.config['uploadFolder'], file.filename))

    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(host = '0.0.0.0',debug=True)

In [ ]:
7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
import os

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///items.db'  # SQLite database file
db = SQLAlchemy(app)

class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(255), nullable=False)

with app.app_context():
    db.create_all()

@app.route('/')
def index():
    items = Item.query.all()
    return render_template('index.html', items=items)

@app.route('/add', methods=['POST'])
def add():
    if request.method == 'POST':
        item_name = request.form['item_name']
        new_item = Item(name=item_name)
        db.session.add(new_item)
        db.session.commit()
    return redirect(url_for('index'))

@app.route('/delete/<int:item_id>')
def delete(item_id):
    item_to_delete = Item.query.get_or_404(item_id)
    db.session.delete(item_to_delete)
    db.session.commit()
    return redirect(url_for('index'))

@app.route('/update/<int:item_id>', methods=['GET', 'POST'])
def update(item_id):
    item_to_update = Item.query.get_or_404(item_id)
    if request.method == 'POST':
        item_to_update.name = request.form['item_name']
        db.session.commit()
        return redirect(url_for('index'))
    return render_template('update.html', item=item_to_update)



if __name__ == '__main__':
    app.run(host='0.0.0.0')

In [ ]:
8. Implement user authentication and registration in a Flask app using Flask-Login.

In [ ]:
from flask import Flask, render_template, redirect, url_for, flash
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired, Length, EqualTo
import os

app = Flask(__name__)
app.secret_key = os.urandom(24)
login_manager = LoginManager(app)
login_manager.login_view = 'login'

# Replace this with your user database or ORM model
class User(UserMixin):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

# Replace this with your user database or ORM model
users = [
    User(1, 'user1', 'password1'),
    User(2, 'user2', 'password2')
]

@login_manager.user_loader
def load_user(user_id):
    return next((user for user in users if user.id == int(user_id)), None)

class LoginForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('Login')

class RegistrationForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired(), Length(min=4, max=20)])
    password = PasswordField('Password', validators=[DataRequired(), Length(min=6, max=20)])
    confirm_password = PasswordField('Confirm Password', validators=[DataRequired(), EqualTo('password')])
    submit = SubmitField('Register')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = next((user for user in users if user.username == form.username.data and user.password == form.password.data), None)
        if user:
            login_user(user)
            flash('Login successful!', 'success')
            return redirect(url_for('dashboard'))
        else:
            flash('Login unsuccessful. Please check your username and password.', 'danger')
    return render_template('login.html', form=form)

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

@app.route('/dashboard')
@login_required
def dashboard():
    return redirect(url_for('home'))

@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegistrationForm()
    if form.validate_on_submit():
        new_user = User(id=len(users) + 1, username=form.username.data, password=form.password.data)
        users.append(new_user)
        flash('Registration successful! You can now log in.', 'success')
        return redirect(url_for('login'))
    return render_template('register.html', form=form)

if __name__ == '__main__':
    app.run(host = '0.0.0.0',debug=True)

In [ ]:
9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)

# Sample data (you can replace this with a database)
books = [
    {"id": 1, "title": "Book 1", "author": "Author 1"},
    {"id": 2, "title": "Book 2", "author": "Author 2"},
    {"id": 3, "title": "Book 3", "author": "Author 3"}
]

@app.route('/')
def index():
    return render_template('index.html', books=books)

@app.route('/add', methods=['GET', 'POST'])
def add_book():
    if request.method == 'POST':
        title = request.form['title']
        author = request.form['author']
        new_book = {"id": len(books) + 1, "title": title, "author": author}
        books.append(new_book)
        return redirect(url_for('index'))
    return render_template('add.html')

@app.route('/edit/<int:book_id>', methods=['GET', 'POST'])
def edit_book(book_id):
    book = next((b for b in books if b['id'] == book_id), None)
    if request.method == 'POST':
        book['title'] = request.form['title']
        book['author'] = request.form['author']
        return redirect(url_for('index'))
    return render_template('edit.html', book=book)

@app.route('/delete/<int:book_id>')
def delete_book(book_id):
    global books
    books = [b for b in books if b['id'] != book_id]
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(host='0.0.0.0',debug=True)

In [ ]:
10. Design a Flask app with proper error handling for 404 and 500 errors.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

# Sample route to demonstrate 404 error
@app.route('/')
def index():
    return render_template('index.html')

# Custom error handler for 404 (Not Found) error
@app.errorhandler(404)
def page_not_found(error):
    return render_template('404.html'), 404

# Custom error handler for 500 (Internal Server Error) error
@app.errorhandler(500)
def internal_server_error(error):
    return render_template('500.html'), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0',debug=True)

# Real-time Development:

In [ ]:
11. Create a real-time chat application using Flask-SocketIO.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('message')
def handle_message(data):
    username = data['username']
    message = data['message']
    print(f'{username}: {message}')
    socketio.emit('message', {'username': username, 'message': message})

if __name__ == '__main__':
    socketio.run(app, debug=True)

In [ ]:
12. Build a Flask app that updates data in real-time using WebSocket connections.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO

app = Flask(__name__)
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('update_data')
def handle_update_data(data):
    # Broadcast the data to all connected clients
    socketio.emit('data_updated', data)

# Added a new route to manually trigger updates
@app.route('/trigger_update')
def trigger_update():
    # Manually trigger an update with sample data
    sample_data = "This is a manual update from the server!"
    socketio.emit('data_updated', sample_data)
    return 'Update triggered successfully!'

if __name__ == '__main__':
    # Run the application with SocketIO support
    socketio.run(app, debug=True)

In [ ]:
13. Implement notifications in a Flask app using websockets to notify users of updates.

In [ ]:
from flask import Flask, render_template, request
from flask_socketio import SocketIO

app = Flask(__name__)
socketio = SocketIO(app)

# Dictionary to store connected clients
connected_clients = {}

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('connect')
def handle_connect():
    sid = request.sid
    print(f'Client connected: {sid}')

    # Store the client's SID in the dictionary
    connected_clients[sid] = True

@socketio.on('disconnect')
def handle_disconnect():
    sid = request.sid
    print(f'Client disconnected: {sid}')

    # Remove the client's SID from the dictionary
    if sid in connected_clients:
        del connected_clients[sid]

@socketio.on('notify')
def handle_notify(data):
    recipient_sid = data['recipient_sid']
    message = data['message']

    # Check if the recipient exists before sending the notification
    if recipient_sid in connected_clients:
        # Send the notification to the specified recipient
        socketio.emit('notification', {'message': message}, room=recipient_sid)
    else:
        print(f'Recipient {recipient_sid} not found.')

if __name__ == '__main__':
    socketio.run(app, debug=False)